# Creating a Convolutional Neural Network for Image Classification 

In this notebook, we'll learn how to create a convolutional neural network for multi-categorical image classification using the FashionMNIST dataset. Almost all of this notebook will look identical to the previous chapter on a non-convlutional neural network except for the new model being introduced. Because of this, we won't spend as much time explaining the identical code.


Here we import our libraries.

In [1]:
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.preprocessing import OneHotEncoder

from IPython.core.display import HTML, display
# Initializing D3 for the entire notebook (this fixes error requiring kernel to load twice)
# (feel free to move this line to anywhere in notebook before visualizations)
display(HTML('<script data-require="d3@3.5.3" data-semver="3.5.3" src="//cdnjs.cloudflare.com/ajax/libs/d3/3.5.3/d3.js"></script>'))

## Processing data 

We utilize the built in training/test set split and reshape our images to be identical vectors. Afterwards, we normalize the image RGB values between 0 and 1. Lastly, we encode our labels to be binary 0 or 1 corresponding to their respective category.

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist

# 60,000 training images, 10,000 test images
(training_imgs, training_labels), (test_imgs, test_labels) = fashion_mnist.load_data()

In [3]:
training_imgs = training_imgs.reshape((training_imgs.shape[0], 28, 28, 1))
training_imgs = training_imgs/255.0
test_imgs = test_imgs/255.0

In [4]:

test_imgs = test_imgs.reshape((test_imgs.shape[0], 28, 28, 1))

In [5]:
encoder1 = OneHotEncoder(sparse=False)
encoder2 = OneHotEncoder(sparse=False)
training_labels = training_labels.reshape(-1, 1)
test_labels = test_labels.reshape(-1, 1)
training_labels = encoder1.fit_transform(training_labels)
test_labels = encoder2.fit_transform(test_labels)

## Building our model

This neural network looks quite different from our previous model, although many factors do stay the same. You'll notice that there are two new elements: a convolutional layer (Conv2D) and a max pooling layer (MaxPooling2D). 

Looking at the model summary, you can start to piece together what each of these layers accomplish. 

A convolutional layer applies a filter to an image to build a new image by applying that filter. This helps us extract relevant information from a picture and discard any information that is not relevant to our image classification problem.

`tf.keras.layers.Conv2D(32, (3,3), activation='relu',  input_shape=(28,28,1))` translates to: apply 32 output filters in our convolution, use a `3x3` convolution window, a `ReLU` activation function, and use our images with an input of `28x28` pixels. 

A max pooling layer works to reduce the dimensions (pixels) of our input images. With a filter size of `2x2`, and a default stride of 2 (stride affects how many pixels are skipped over when our filter looks at an input image), our output dimensions will be cut in half `(None, 26, 26, 32)` to  `(None, 13, 13, 32)`. We keep the results from our filtered image and perhaps enhance relevant details. Because of this, max pooling is useful to simplify computation and reduce overfitting.



In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu',  input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')

])

In [7]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               102528    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1

## Training our model 

Now we can train our model. How does this convolutional model compare to the standard neural network model that we creatd in the previous chapter?

In [8]:
# Importing callback data extractor
import os, sys
import libraries.extractioncallback as excb
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

extractor = excb.CallbackDataExtractor(
    model = model,
    layer = 1,
    validation_data = (test_imgs, test_labels),
    sample_every = 1,
    rec_int_values = False,
    is_bin = False
)

In [ ]:
hist = model.fit(training_imgs, training_labels, verbose=1, epochs=10, validation_split=0.3, callbacks=[extractor])

Epoch 1/10
1313/1313 [==============================] - 11s 8ms/step - loss: 0.9979 - accuracy: 0.6470 - val_loss: 0.6986 - val_accuracy: 0.7455
Epoch 2/10
1313/1313 [==============================] - 10s 8ms/step - loss: 0.6001 - accuracy: 0.7737 - val_loss: 0.5864 - val_accuracy: 0.7867
Epoch 3/10
1313/1313 [==============================] - 10s 8ms/step - loss: 0.5149 - accuracy: 0.8088 - val_loss: 0.5622 - val_accuracy: 0.7627
Epoch 4/10
1313/1313 [==============================] - 11s 8ms/step - loss: 0.4676 - accuracy: 0.8280 - val_loss: 0.5175 - val_accuracy: 0.8058
Epoch 5/10
1313/1313 [==============================] - 11s 8ms/step - loss: 0.4345 - accuracy: 0.8418 - val_loss: 0.4478 - val_accuracy: 0.8343
Epoch 6/10
1313/1313 [==============================] - 10s 8ms/step - loss: 0.4099 - accuracy: 0.8520 - val_loss: 0.4154 - val_accuracy: 0.8502
Epoch 7/10
1313/1313 [==============================] - 10s 8ms/step - loss: 0.3887 - accuracy: 0.8600 - val_loss: 0.4038 - val_ac

We can look at the model's accuracy and loss per epoch below.

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

## Evaluating our model

The process for evaluating our model follows from previous notebooks. We will see the model's confidence score in its predictions on the test set, print a classification report, then plot a few results from the test set.

In [ ]:
from sklearn.metrics import classification_report

testingPredictions = model.predict(test_imgs)
testingPredictions = list(testingPredictions.argmax(axis=-1))
confidence_scores = model.predict(test_imgs, batch_size=32)
np.set_printoptions(suppress=True)
print(confidence_scores)


In [ ]:
from sklearn.metrics import classification_report

target_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print(classification_report(test_labels.argmax(axis=-1), testingPredictions, target_names=target_names))

In [ ]:
def plot_image(i, predictions_array, confidence_scores, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)


  #predicted_label = np.argmax(predictions_array)
  #print(predicted_label, true_label)
  if predictions_array[i] == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(target_names[predictions_array[i]],
                                100*np.max(confidence_scores[i]),
                                target_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
num_rows = 3
num_cols = 5
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
vec = np.vectorize(confidence_scores)
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, testingPredictions, confidence_scores, test_labels.argmax(axis=-1), test_imgs)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, list(confidence_scores)[i], test_labels.argmax(axis=-1))
plt.tight_layout()
plt.show()

Now, we need to export the data in order to support some interactive visualizations that we've created. Feel free to skip over this code block and move to the interactive visualizations below.

In [ ]:
import json, os, sys

output_directory = "libraries/stored_results"
output_filename = "c_neural_network_img.json"
full_path = os.path.join(output_directory, output_filename)

epoch_output = extractor.get_testing_results()
extractor.generateJSON(num_entries=2000, path=full_path)

In [ ]:
import libraries.mlvislib as mlvs
cm = mlvs.ConfusionMatrix(full_path, x_labels=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"],
                          y_labels=["9", "8", "7", "6", "7", "6", "5", "4", "3", "2", "1", "0"])
cm.display()